In [3]:
import torch
from torch import Tensor, nn 
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
from sklearn.datasets import load_boston
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [4]:
X, y = load_boston(return_X_y = True)
print(f"X shape : {X.shape},  y shape : {y.shape}")

X shape : (506, 13),  y shape : (506,)


c:\Users\rubin\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housin

In [7]:
class SklearnDataset(Dataset):
    
    def __init__(self, X:np.ndarray, y:np.ndarray):
        super().__init__()
        scaler = MinMaxScaler()
        scaler.fit(X)
        self.X = scaler.transform(X)
        self.Y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        x = self.X[idx].astype(np.float32)
        y = self.Y[idx].astype(np.float32)
        return x, y

In [11]:
class LinRegModel(pl.LightningModule):
    
    def __init__(self, input_dim:int):
        super().__init__()
        self.linear = nn.Linear(in_features=13, out_features=1, bias=True)
        
    def forward(self, x):
        y_hat = self.linear(x)
        return y_hat
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        x = x.view(x.size(0), -1)
        y_hat = self.forward(x)
        loss = F.mse_loss(y_hat, y, reduction='sum')
        return loss
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-4)

In [12]:
bostonds = SklearnDataset(X, y)
train_loader = DataLoader(bostonds, batch_size=32, shuffle=True, drop_last=True)

In [13]:
trainer = pl.Trainer()
model = LinRegModel(input_dim=13)
trainer.fit(model, train_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\rubin\anaconda3\envs\tf_gpu\lib\site-packages\pytorch_lightning\loops\utilities.py:70: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
Missing logger folder: e:\딥러닝파이토치교과서\PyTorch_DL_ChatBot\lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type   | Params
----------------------------------
0 | linear | Linear | 14    
----------------------------------
14        Trainable params
0         Non-trainable params
14        Total params
0.000     Total estimated model params size (MB)
c:\Users\rubin\anaconda3\envs\tf_gpu\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider i

Epoch 1:   0%|          | 0/15 [00:00<?, ?it/s, v_num=0]          

C:\Users\rubin\AppData\Local\Temp\ipykernel_23756\1621289588.py:15: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(y_hat, y, reduction='sum')


Epoch 999: 100%|██████████| 15/15 [00:00<00:00, 178.57it/s, v_num=0]

`Trainer.fit` stopped: `max_epochs=1000` reached.


Epoch 999: 100%|██████████| 15/15 [00:00<00:00, 164.83it/s, v_num=0]
